https://medium.com/mobius-labs/accelerating-large-language-models-strategies-for-enhancing-your-ai-inference-speed-a77dddc33bb2#:~:text=In%20this%20blog%20post%2C%20we%E2%80%99ll%20delve%20into%20various,and%20pruning%20to%20knowledge%20distillation%20and%20batch%20inference.

In [3]:
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

# تحميل نموذج BERT مسبق التدريب
model = BertModel.from_pretrained('bert-base-uncased')

# تحديد طبقة معينة لتطبيق التقطير عليها
layer = model.encoder.layer[0].attention.self.query

# تطبيق التقطير باستخدام طريقة L1 unstructured لإزالة 20% من الأوزان
prune.l1_unstructured(layer, name='weight', amount=0.2)

# طباعة الأوزان بعد التقطير (سيتم تعيين الأوزان المقطرة إلى صفر)
print(layer.weight)

# جعل التقطير دائمًا عن طريق إزالة قناع التقطير
prune.remove(layer, 'weight')

# طباعة الأوزان بعد إزالة القناع (سيتم إزالة الأوزان المقطرة نهائيًا)
print(layer.weight)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       grad_fn=<MulBackward0>)
Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       requires_grad=True)


In [4]:
import torch.quantization

# تطبيق التكمية الديناميكية على النموذج
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [5]:
import torch
import torch.nn as nn
from transformers import BertModel, BertForSequenceClassification, AdamW

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # افترض تصنيف ثنائي

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# تحميل نموذج المعلم (نموذج كبير مسبق التدريب)
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

# إنشاء نموذج الطالب
student_model = StudentModel()

# افترض أن input_data, attention_mask, و labels هي بيانات الإدخال
input_data = torch.tensor([...])  # استبدل ببيانات فعلية
attention_mask = torch.tensor([...])  # استبدل ببيانات فعلية
labels = torch.tensor([...])  # استبدل ببيانات فعلية

# تمرير البيانات عبر نموذج المعلم
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# تمرير البيانات عبر نموذج الطالب
student_logits = student_model(input_data, attention_mask=attention_mask)

# حساب فقد التقطير (Distillation Loss)
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# حساب فقد التصنيف (Classification Loss)
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# الفقد الكلي
loss = distillation_loss + classification_loss

# تحديث الأوزان باستخدام AdamW
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Could not infer dtype of ellipsis

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.rnn = nn.RNN(input_size=200, hidden_size=1000, num_layers=1)
        self.linear = nn.Linear(1000, 100)

    def forward(self, x, init):
        x = self.rnn(x, init)[0]
        y = self.linear(x.view(x.size(0)*x.size(1), x.size(2)))
        return y.view(x.size(0), x.size(1), y.size(1))

net = testNet()
init = torch.zeros(1, 4, 1000)  # تهيئة الحالة الأولية
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

total_loss = 0.0
for i in range(10000):  # 10000 دُفعة صغيرة
    input = torch.randn(1000, 4, 200)  # بيانات عشوائية
    target = torch.LongTensor(4, 1000).zero_()  # أهداف عشوائية

    optimizer.zero_grad()
    output = net(input, init)
    loss = criterion(output.view(-1, output.size(2)), target.view(-1))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()  # جمع الفقد

print(total_loss)

In [6]:
!pip install mem_usage_ui
!pipx install asciinema

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.7/756.7 kB 18.7 MB/s eta 0:00:00
/bin/bash: line 1: pipx: command not found


In [7]:
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel, BertForSequenceClassification, AdamW
import torch.nn as nn
import torch.nn.functional as F

# Load a pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Select a layer to prune
layer = model.encoder.layer[0].attention.self.query

# Use the `l1_unstructured` method to prune 20% of the connections in the layer
prune.l1_unstructured(layer, name='weight', amount=0.2)

# The pruned entries in the weight tensor are now set to zero
print(layer.weight)

# To make the pruning permanent, remove the 'weight_mask' parameter
prune.remove(layer, 'weight')

# The pruned entries are now actually removed from the weight tensor
print(layer.weight)

# Dynamic quantization for the model
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Define the student model
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# Assume teacher_model is your pre-trained large language model
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# Assume input_data, attention_mask, and labels are your input tensors
input_data = torch.tensor([[101, 102, 103, 104, 105]])  # Replace with actual data
attention_mask = torch.tensor([[1, 1, 1, 1, 1]])  # Replace with actual data
labels = torch.tensor([0])  # Replace with actual data

# Forward pass through the teacher model
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# Forward pass through the student model
student_logits = student_model(input_data, attention_mask=attention_mask)

# Calculate the distillation loss
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# You might also want to add a classification loss term
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# Total loss
loss = distillation_loss + classification_loss

# Backward pass and optimization
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()

tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       grad_fn=<MulBackward0>)
Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       requires_grad=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:3369: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
!rm -rf /root/.cache

### تقطير المعرفة نقل معرفة من نموذج اكبر لنموذج اصغر

In [1]:
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

# تحميل نموذج BERT مسبق التدريب
model = BertModel.from_pretrained('bert-base-uncased')

# تحديد طبقة معينة لتطبيق التقطير عليها
layer = model.encoder.layer[0].attention.self.query

# تطبيق التقطير باستخدام طريقة L1 unstructured لإزالة 20% من الأوزان
prune.l1_unstructured(layer, name='weight', amount=0.2)

# طباعة الأوزان بعد التقطير (سيتم تعيين الأوزان المقطرة إلى صفر)
print(layer.weight)

# جعل التقطير دائمًا عن طريق إزالة قناع التقطير
prune.remove(layer, 'weight')

# طباعة الأوزان بعد إزالة القناع (سيتم إزالة الأوزان المقطرة نهائيًا)
print(layer.weight)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       grad_fn=<MulBackward0>)
Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       requires_grad=True)


In [2]:
import torch.quantization

# تطبيق التكمية الديناميكية على النموذج
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.rnn = nn.RNN(input_size=200, hidden_size=1000, num_layers=1)
        self.linear = nn.Linear(1000, 100)

    def forward(self, x, init):
        x = self.rnn(x, init)[0]
        y = self.linear(x.view(x.size(0)*x.size(1), x.size(2)))
        return y.view(x.size(0), x.size(1), y.size(1))

net = testNet()
init = torch.zeros(1, 4, 1000)  # تهيئة الحالة الأولية
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

total_loss = 0.0
for i in range(10000):  # 10000 دُفعة صغيرة
    input = torch.randn(1000, 4, 200)  # بيانات عشوائية
    target = torch.LongTensor(4, 1000).zero_()  # أهداف عشوائية

    optimizer.zero_grad()
    output = net(input, init)
    loss = criterion(output.view(-1, output.size(2)), target.view(-1))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()  # جمع الفقد

print(total_loss)

KeyboardInterrupt: 

التحقق من نقل المعرفة

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertForSequenceClassification, AdamW

# 1. تحقق من التقطير (Pruning)
def verify_pruning(model, layer_path):
    # تحديد الطبقة المطلوبة
    layer = eval(f"model.{layer_path}")

    # التأكد من وجود أوزان مقطرة (أوزان بقيمة صفر)
    pruned_weights = (layer.weight == 0).sum().item()
    total_weights = layer.weight.numel()

    print(f"عدد الأوزان المقطرة: {pruned_weights} من أصل {total_weights}")
    print(f"نسبة التقطير: {pruned_weights / total_weights * 100:.2f}%")

# 2. تحقق من نقل المعرفة (Knowledge Distillation)
def verify_distillation(teacher_model, student_model, input_data, attention_mask):
    # وضع النموذجين في وضع التقييم (Evaluation Mode)
    teacher_model.eval()
    student_model.eval()

    # تمرير البيانات عبر نموذج المعلم
    with torch.no_grad():
        teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

    # تمرير البيانات عبر نموذج الطالب
    student_logits = student_model(input_data, attention_mask=attention_mask)

    # حساب التشابه بين خرج النموذجين باستخدام Cosine Similarity
    cosine_sim = F.cosine_similarity(teacher_logits.flatten(), student_logits.flatten(), dim=0).item()

    print(f"التشابه بين خرج النموذجين (Cosine Similarity): {cosine_sim:.4f}")

# 3. إنشاء نموذج الطالب
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # افترض تصنيف ثنائي

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# 4. تحميل النماذج
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# 5. بيانات عشوائية للاختبار
input_data = torch.tensor([[101, 102, 103, 104, 105]])  # بيانات عشوائية (يجب استبدالها ببيانات فعلية)
attention_mask = torch.tensor([[1, 1, 1, 1, 1]])  # قناع الانتباه (يجب استبدالها ببيانات فعلية)

# 6. تطبيق التقطير على نموذج الطالب
import torch.nn.utils.prune as prune
layer_to_prune = "bert.encoder.layer[0].attention.self.query"
prune.l1_unstructured(eval(f"student_model.{layer_to_prune}"), name='weight', amount=0.2)

# 7. التحقق من التقطير
print("التحقق من التقطير:")
verify_pruning(student_model, layer_to_prune)

# 8. التحقق من نقل المعرفة
print("\nالتحقق من نقل المعرفة:")
verify_distillation(teacher_model, student_model, input_data, attention_mask)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


التحقق من التقطير:
عدد الأوزان المقطرة: 117965 من أصل 589824
نسبة التقطير: 20.00%

التحقق من نقل المعرفة:
التشابه بين خرج النموذجين (Cosine Similarity): 0.8759




تصحيح كود التقطير ليعمل على كولاب
استخدام التقطير
صحح الاكواد لتعمل ف كوةلاب على المعالج
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

# Load a pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Select a layer to prune
layer = model.encoder.layer[0].attention.self.query

# Use the `l1_unstructured` method to prune 20% of the connections in the layer
prune.l1_unstructured(layer, 'weight', amount=0.2)

# The pruned entries in the weight tensor are now set to zero
print(layer.weight)

# To make the pruning permanent, remove the 'weight_mask' parameter
prune.remove(layer, 'weight')

# The pruned entries are now actually removed from the weight tensor
print(layer.weight)

import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}

import torch
import torch.nn as nn
from transformers import BertModel, BertForSequenceClassification, AdamW

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# Assume teacher_model is your pre-trained large language model
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# Assume input_data, attention_mask, and labels are your input tensors
input_data = torch.tensor([...])  # Replace with actual data
attention_mask = torch.tensor([...])  # Replace with actual data
labels = torch.tensor([...])  # Replace with actual data

# Forward pass through the teacher model
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# Forward pass through the student model
student_logits = student_model(input_data, attention_mask=attention_mask)

# Calculate the distillation loss
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# You might also want to add a classification loss term
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# Total loss
loss = distillation_loss + classification_loss

# Backward pass and optimization
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()
لتصحيح الكود وجعله يعمل بشكل صحيح في بيئة كولاب (Colab) على المعالج (CPU)، إليك النسخة المعدلة من الكود مع بعض التعديلات والتعليقات التوضيحية:

python
Copy
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel, BertForSequenceClassification, AdamW
import torch.nn as nn
import torch.nn.functional as F

# Load a pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Select a layer to prune
layer = model.encoder.layer[0].attention.self.query

# Use the `l1_unstructured` method to prune 20% of the connections in the layer
prune.l1_unstructured(layer, name='weight', amount=0.2)

# The pruned entries in the weight tensor are now set to zero
print(layer.weight)

# To make the pruning permanent, remove the 'weight_mask' parameter
prune.remove(layer, 'weight')

# The pruned entries are now actually removed from the weight tensor
print(layer.weight)

# Dynamic quantization for the model
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Define the student model
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# Assume teacher_model is your pre-trained large language model
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# Assume input_data, attention_mask, and labels are your input tensors
input_data = torch.tensor([[101, 102, 103, 104, 105]])  # Replace with actual data
attention_mask = torch.tensor([[1, 1, 1, 1, 1]])  # Replace with actual data
labels = torch.tensor([0])  # Replace with actual data

# Forward pass through the teacher model
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# Forward pass through the student model
student_logits = student_model(input_data, attention_mask=attention_mask)

# Calculate the distillation loss
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# You might also want to add a classification loss term
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# Total loss
loss = distillation_loss + classification_loss

# Backward pass and optimization
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()
التعديلات الرئيسية:
التكمية الديناميكية: تم تصحيح استدعاء torch.quantization.quantize_dynamic بإضافة dtype=torch.qint8 لتحديد نوع البيانات المطلوب للتكمية.

تنسيق البيانات: تم إضافة بيانات وهمية لـ input_data و attention_mask و labels لتوضيح كيفية استخدامها. يجب استبدال هذه البيانات بالبيانات الفعلية الخاصة بك.

تصحيح الأخطاء النحوية: تم تصحيح بعض الأخطاء النحوية مثل quantize_dynamic(model, {torch.nn.Linear} إلى quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8).

التعليقات التوضيحية: تم إضافة تعليقات توضيحية لتسهيل فهم الكود.

ملاحظة:
تأكد من تثبيت الحزم المطلوبة مثل torch و transformers قبل تشغيل الكود.

يمكنك تشغيل هذا الكود في بيئة كولاب (Colab) أو أي بيئة تدعم تشغيل نماذج PyTorch.

إذا كنت بحاجة إلى مزيد من التوضيحات أو تعديلات إضافية، فلا تتردد في طلب ذلك!

ما معنى


[ ]

Start coding or generate with AI.

[ ]

import torch
import torch.nn as nn



def __init__(self, num_classes, input_size, hidden_size, num_layers):
    super(LSTM, self).__init__()
    
    self.num_classes = num_classes
    self.num_layers = num_layers
    self.input_size = input_size
    self.hidden_size = hidden_size
    #self.seq_length = seq_length
    self.dropout = nn.Dropout(p=0.2)
    
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                        num_layers=num_layers, batch_first=True,dropout = 0.25)
    
    self.fc = nn.Linear(hidden_size, num_classes)

def forward(self, x):
    h_0 = Variable(torch.zeros(
        self.num_layers, x.size(0), self.hidden_size).to(device))
    
    c_0 = Variable(torch.zeros(
        self.num_layers, x.size(0), self.hidden_size).to(device))
    
    # Propagate input through LSTM
    ula, (h_out, _) = self.lstm(x, (h_0, c_0))
    
    h_out = h_out.view(-1, self.hidden_size)
    
    out = self.fc(h_out)
    out = self.dropout(out)
   
    return out

[ ]
import torch
import torch.nn as nn
class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.rnn = nn.RNN(input_size=200, hidden_size=1000, num_layers=1)
        self.linear = nn.Linear(1000, 100)

    def forward(self, x, init):
        x = self.rnn(x, init)[0]
        y = self.linear(x.view(x.size(0)*x.size(1), x.size(2)))
        return y.view(x.size(0), x.size(1), y.size(1))

net = testNet()
init = Variable(torch.zeros(1, 4, 1000))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

total_loss = 0.0
for i in range(10000): #10000 mini-batch
    input = Variable(torch.randn(1000, 4, 200)) #Seqlen = 1000, batch_size = 4, feature = 200
    target = Variable(torch.LongTensor(4, 1000).zero_())

    optimizer.zero_grad()
    output = net(input, init)
    loss = criterion(output.view(-1, output.size(2)), target.view(-1))
    loss.backward()
    optimizer.step()
    total_loss += loss[0]

print(total_loss)

Next steps:

[ ]
set term png small size 800,600
set output "mem-graph.png"

set ylabel "VSZ"
set y2label "%MEM"

set ytics nomirror
set y2tics nomirror in

set yrange [0:*]
set y2range [0:*]

plot "/tmp/mem.log" using 3 with lines axes x1y1 title "VSZ", \
     "/tmp/mem.log" using 2 with lines axes x1y2 title "%MEM"

[ ]
!pip install mem_usage_ui
Collecting mem_usage_ui
  Downloading mem_usage_ui-1.0.9-py3-none-any.whl.metadata (1.8 kB)
Requirement already satisfied: aiohttp<5,>=3 in /usr/local/lib/python3.11/dist-packages (from mem_usage_ui) (3.11.11)
Requirement already satisfied: psutil<6 in /usr/local/lib/python3.11/dist-packages (from mem_usage_ui) (5.9.5)
Requirement already satisfied: aiohappyeyeballs>=2.3.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (2.4.4)
Requirement already satisfied: aiosignal>=1.1.2 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (1.3.2)
Requirement already satisfied: attrs>=17.3.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (24.3.0)
Requirement already satisfied: frozenlist>=1.1.1 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (1.5.0)
Requirement already satisfied: multidict<7.0,>=4.5 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (6.1.0)
Requirement already satisfied: propcache>=0.2.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (0.2.1)
Requirement already satisfied: yarl<2.0,>=1.17.0 in /usr/local/lib/python3.11/dist-packages (from aiohttp<5,>=3->mem_usage_ui) (1.18.3)
Requirement already satisfied: idna>=2.0 in /usr/local/lib/python3.11/dist-packages (from yarl<2.0,>=1.17.0->aiohttp<5,>=3->mem_usage_ui) (3.10)
Downloading mem_usage_ui-1.0.9-py3-none-any.whl (756 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.7/756.7 kB 8.6 MB/s eta 0:00:00
Installing collected packages: mem_usage_ui
Successfully installed mem_usage_ui-1.0.9

[ ]
!pipx install asciinema
/bin/bash: line 1: pipx: command not found

[ ]

Start coding or generate with AI.

[ ]
import torch
import torch.nn as nn
import torch.optim as optim # Import optim module

class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.rnn = nn.RNN(input_size=200, hidden_size=1000, num_layers=1)
        self.linear = nn.Linear(1000, 100)

    def forward(self, x, init):
        x = self.rnn(x, init)[0]
        y = self.linear(x.view(x.size(0)*x.size(1), x.size(2)))
        return y.view(x.size(0), x.size(1), y.size(1))

net = testNet()
init = torch.zeros(1, 4, 1000) # Removed Variable and use tensor directly
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) # Use optim

total_loss = 0.0
for i in range(10000): #10000 mini-batch
    input = torch.randn(1000, 4, 200) # Removed Variable and use tensor directly
    target = torch.LongTensor(4, 1000).zero_() # Removed Variable and use tensor directly

    optimizer.zero_grad()
    output = net(input, init)
    loss = criterion(output.view(-1, output.size(2)), target.view(-1))
    loss.backward()
    optimizer.step()
    total_loss += loss.item() # Changed loss[0] to loss.item() to get the loss value

print(total_loss)


[ ]
import torch
import torch.nn as nn
from transformers import BertModel, BertForSequenceClassification, AdamW

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits
        

teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()
input_data = torch.tensor([...])  # Replace with actual data
attention_mask = torch.tensor([...])  # Replace with actual data
labels = torch.tensor([...])  # Replace with actual data

with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

student_logits = student_model(input_data, attention_mask=attention_mask)

distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

loss = distillation_loss + classification_loss

optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()

Next steps:
https://medium.com/mobius-labs/accelerating-large-language-models-strategies-for-enhancing-your-ai-inference-speed-a77dddc33bb2#:~:text=In%20this%20blog%20post%2C%20we%E2%80%99ll%20delve%20into%20various,and%20pruning%20to%20knowledge%20distillation%20and%20batch%20inference.


[ ]
import torch
import torch.nn as nn
from transformers import BertModel, BertForSequenceClassification, AdamW

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits
        

teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()
input_data = torch.tensor([...])  # Replace with actual data
attention_mask = torch.tensor([...])  # Replace with actual data
labels = torch.tensor([...])  # Replace with actual data

with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

student_logits = student_model(input_data, attention_mask=attention_mask)

distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

loss = distillation_loss + classification_loss

optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()

Next steps:

[ ]
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

# Load a pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Select a layer to prune
layer = model.encoder.layer[0].attention.self.query

# Use the `l1_unstructured` method to prune 20% of the connections in the layer
prune.l1_unstructured(layer, 'weight', amount=0.2)

# The pruned entries in the weight tensor are now set to zero
print(layer.weight)

# To make the pruning permanent, remove the 'weight_mask' parameter
prune.remove(layer, 'weight')

# The pruned entries are now actually removed from the weight tensor
print(layer.weight)
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       grad_fn=<MulBackward0>)
Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       requires_grad=True)

[ ]
import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}

Next steps:

[ ]
import torch
import torch.nn as nn
from transformers import BertModel, BertForSequenceClassification, AdamW

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# Assume teacher_model is your pre-trained large language model
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# Assume input_data, attention_mask, and labels are your input tensors
input_data = torch.tensor([...])  # Replace with actual data
attention_mask = torch.tensor([...])  # Replace with actual data
labels = torch.tensor([...])  # Replace with actual data

# Forward pass through the teacher model
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# Forward pass through the student model
student_logits = student_model(input_data, attention_mask=attention_mask)

# Calculate the distillation loss
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# You might also want to add a classification loss term
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# Total loss
loss = distillation_loss + classification_loss

# Backward pass and optimization
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()

Next steps:
Double-click (or enter) to edit


[ ]

Start coding or generate with AI.

[ ]

Start coding or generate with AI.

[1]
22s
optimizer.step()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       grad_fn=<MulBackward0>)
Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       requires_grad=True)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:3369: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
الكود الذي قدمته يحتوي على عدة أجزاء تعليمية وتطبيقية تتعلق بتعلم الآلة باستخدام مكتبة PyTorch ومكتبة Hugging Face Transformers. سأشرح كل جزء بشكل مفصل:

1. التقطير (Pruning) للنموذج
python
Copy
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

# تحميل نموذج BERT مسبق التدريب
model = BertModel.from_pretrained('bert-base-uncased')

# تحديد طبقة معينة لتطبيق التقطير عليها
layer = model.encoder.layer[0].attention.self.query

# تطبيق التقطير باستخدام طريقة L1 unstructured لإزالة 20% من الأوزان
prune.l1_unstructured(layer, name='weight', amount=0.2)

# طباعة الأوزان بعد التقطير (سيتم تعيين الأوزان المقطرة إلى صفر)
print(layer.weight)

# جعل التقطير دائمًا عن طريق إزالة قناع التقطير
prune.remove(layer, 'weight')

# طباعة الأوزان بعد إزالة القناع (سيتم إزالة الأوزان المقطرة نهائيًا)
print(layer.weight)
التفسير:

التقطير (Pruning): تقنية تستخدم لتقليل حجم النموذج عن طريق إزالة الأوزان الأقل أهمية.

هنا يتم تطبيق التقطير على طبقة معينة من نموذج BERT باستخدام طريقة l1_unstructured.

بعد التقطير، يتم تعيين الأوزان الأقل أهمية إلى صفر.

prune.remove يجعل التقطير دائمًا عن طريق إزالة الأوزان المقطرة نهائيًا.

2. التكمية الديناميكية (Dynamic Quantization)
python
Copy
import torch.quantization

# تطبيق التكمية الديناميكية على النموذج
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
التفسير:

التكمية (Quantization): تقنية تستخدم لتقليل حجم النموذج وتحسين أدائه عن طريق تقليل دقة الأوزان (مثل تحويلها من 32 بت إلى 8 بت).

هنا يتم تطبيق التكمية الديناميكية على طبقات Linear في النموذج.

3. نموذج الطالب (Student Model) ونموذج المعلم (Teacher Model)
python
Copy
import torch
import torch.nn as nn
from transformers import BertModel, BertForSequenceClassification, AdamW

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # افترض تصنيف ثنائي

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# تحميل نموذج المعلم (نموذج كبير مسبق التدريب)
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

# إنشاء نموذج الطالب
student_model = StudentModel()

# افترض أن input_data, attention_mask, و labels هي بيانات الإدخال
input_data = torch.tensor([...])  # استبدل ببيانات فعلية
attention_mask = torch.tensor([...])  # استبدل ببيانات فعلية
labels = torch.tensor([...])  # استبدل ببيانات فعلية

# تمرير البيانات عبر نموذج المعلم
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# تمرير البيانات عبر نموذج الطالب
student_logits = student_model(input_data, attention_mask=attention_mask)

# حساب فقد التقطير (Distillation Loss)
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# حساب فقد التصنيف (Classification Loss)
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# الفقد الكلي
loss = distillation_loss + classification_loss

# تحديث الأوزان باستخدام AdamW
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()
التفسير:

نموذج الطالب والمعلم: تقنية تستخدم لتدريب نموذج صغير (الطالب) على تقليد نموذج كبير (المعلم).

يتم حساب فقد التقطير باستخدام KLDivLoss لمقارنة توزيعات النموذجين.

يتم أيضًا حساب فقد التصنيف باستخدام CrossEntropyLoss.

يتم تحديث أوزان نموذج الطالب باستخدام مُحسّن AdamW.

4. تدريب نموذج RNN
python
Copy
import torch
import torch.nn as nn
import torch.optim as optim

class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.rnn = nn.RNN(input_size=200, hidden_size=1000, num_layers=1)
        self.linear = nn.Linear(1000, 100)

    def forward(self, x, init):
        x = self.rnn(x, init)[0]
        y = self.linear(x.view(x.size(0)*x.size(1), x.size(2)))
        return y.view(x.size(0), x.size(1), y.size(1))

net = testNet()
init = torch.zeros(1, 4, 1000)  # تهيئة الحالة الأولية
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

total_loss = 0.0
for i in range(10000):  # 10000 دُفعة صغيرة
    input = torch.randn(1000, 4, 200)  # بيانات عشوائية
    target = torch.LongTensor(4, 1000).zero_()  # أهداف عشوائية

    optimizer.zero_grad()
    output = net(input, init)
    loss = criterion(output.view(-1, output.size(2)), target.view(-1))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()  # جمع الفقد

print(total_loss)
التفسير:

يتم تدريب نموذج RNN بسيط باستخدام بيانات عشوائية.

يتم استخدام CrossEntropyLoss لحساب الفقد وSGD لتحديث الأوزان.

5. تثبيت مكتبات إضافية
bash
Copy
!pip install mem_usage_ui
!pipx install asciinema
التفسير:

يتم تثبيت مكتبات إضافية لمراقبة استخدام الذاكرة (mem_usage_ui) وتسجيل الجلسات (asciinema).

6. الخطوات التالية
يتم اقتراح خطوات إضافية مثل تحسين سرعة الاستدلال (Inference Speed) باستخدام تقنيات مثل التكمية، التقطير، والاستدلال بالدفعات (Batch Inference).

إذا كان لديك أي أسئلة إضافية أو تحتاج إلى مزيد من التوضيحات، فلا تتردد في السؤال!

اكتب كود بايثون للتاكد من تحقق عملية التقطير ونقل المعرفة من نموذج للنموذج الاخر
للتأكد من أن عملية التقطير (Pruning) ونقل المعرفة (Knowledge Distillation) تعمل بشكل صحيح، يمكننا كتابة كود بايثون يقوم بما يلي:

التقطير (Pruning): التأكد من أن الأوزان المقطرة تم تعيينها إلى صفر.

نقل المعرفة (Knowledge Distillation): التأكد من أن نموذج الطالب يتعلم من نموذج المعلم بشكل صحيح عن طريق مقارنة الخرج (output) من النموذجين.

إليك الكود الكامل:

الكود:
python
Copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertForSequenceClassification, AdamW

# 1. تحقق من التقطير (Pruning)
def verify_pruning(model, layer_path):
    # تحديد الطبقة المطلوبة
    layer = eval(f"model.{layer_path}")
    
    # التأكد من وجود أوزان مقطرة (أوزان بقيمة صفر)
    pruned_weights = (layer.weight == 0).sum().item()
    total_weights = layer.weight.numel()
    
    print(f"عدد الأوزان المقطرة: {pruned_weights} من أصل {total_weights}")
    print(f"نسبة التقطير: {pruned_weights / total_weights * 100:.2f}%")

# 2. تحقق من نقل المعرفة (Knowledge Distillation)
def verify_distillation(teacher_model, student_model, input_data, attention_mask):
    # وضع النموذجين في وضع التقييم (Evaluation Mode)
    teacher_model.eval()
    student_model.eval()
    
    # تمرير البيانات عبر نموذج المعلم
    with torch.no_grad():
        teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits
    
    # تمرير البيانات عبر نموذج الطالب
    student_logits = student_model(input_data, attention_mask=attention_mask)
    
    # حساب التشابه بين خرج النموذجين باستخدام Cosine Similarity
    cosine_sim = F.cosine_similarity(teacher_logits.flatten(), student_logits.flatten(), dim=0).item()
    
    print(f"التشابه بين خرج النموذجين (Cosine Similarity): {cosine_sim:.4f}")

# 3. إنشاء نموذج الطالب
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # افترض تصنيف ثنائي

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# 4. تحميل النماذج
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# 5. بيانات عشوائية للاختبار
input_data = torch.tensor([[101, 102, 103, 104, 105]])  # بيانات عشوائية (يجب استبدالها ببيانات فعلية)
attention_mask = torch.tensor([[1, 1, 1, 1, 1]])  # قناع الانتباه (يجب استبدالها ببيانات فعلية)

# 6. تطبيق التقطير على نموذج الطالب
import torch.nn.utils.prune as prune
layer_to_prune = "bert.encoder.layer[0].attention.self.query"
prune.l1_unstructured(eval(f"student_model.{layer_to_prune}"), name='weight', amount=0.2)

# 7. التحقق من التقطير
print("التحقق من التقطير:")
verify_pruning(student_model, layer_to_prune)

# 8. التحقق من نقل المعرفة
print("\nالتحقق من نقل المعرفة:")
verify_distillation(teacher_model, student_model, input_data, attention_mask)
شرح الكود:
تحقق من التقطير (Pruning):

يتم تحديد الطبقة التي تم تقطيعها (مثل bert.encoder.layer[0].attention.self.query).

يتم حساب عدد الأوزان المقطرة (الأوزان التي تم تعيينها إلى صفر) ونسبتها إلى إجمالي الأوزان.

تحقق من نقل المعرفة (Knowledge Distillation):

يتم تمرير نفس البيانات عبر نموذج المعلم ونموذج الطالب.

يتم حساب التشابه بين خرج النموذجين باستخدام Cosine Similarity.

إذا كان التشابه قريبًا من 1، فهذا يعني أن نموذج الطالب يتعلم بشكل جيد من نموذج المعلم.

بيانات الاختبار:

يتم استخدام بيانات عشوائية للاختبار (يجب استبدالها ببيانات فعلية).

التقطير:

يتم تطبيق التقطير على طبقة معينة في نموذج الطالب باستخدام prune.l1_unstructured.

مثال على الخرج المتوقع:
Copy
التحقق من التقطير:
عدد الأوزان المقطرة: 1536 من أصل 7680
نسبة التقطير: 20.00%

التحقق من نقل المعرفة:
التشابه بين خرج النموذجين (Cosine Similarity): 0.8765
ملاحظات:
يمكنك تعديل نسبة التقطير (amount) في prune.l1_unstructured لرؤية تأثيرات مختلفة.

إذا كانت قيمة التشابه (Cosine Similarity) قريبة من 1، فهذا يعني أن نقل المعرفة يعمل بشكل جيد.

تأكد من استخدام بيانات فعلية (input_data, attention_mask) للحصول على نتائج دقيقة.

إذا كان لديك أي أسئلة إضافية أو تحتاج إلى مزيد من التوضيحات، فلا تتردد في السؤال!

New chat
Message DeepSeek
AI-generated, for reference only


### نقل المعرفة من نموذج لنموذج

Model Pruning

### Transferring knowledge from one model to another

### 1تنزيل النماذج ونقل المعرفة او التقطير

In [4]:
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel, BertForSequenceClassification, AdamW
import torch.nn as nn
import torch.nn.functional as F

# Load a pre-trained BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Select a layer to prune
layer = model.encoder.layer[0].attention.self.query

# Use the `l1_unstructured` method to prune 20% of the connections in the layer
prune.l1_unstructured(layer, name='weight', amount=0.2)

# The pruned entries in the weight tensor are now set to zero
print(layer.weight)

# To make the pruning permanent, remove the 'weight_mask' parameter
prune.remove(layer, 'weight')

# The pruned entries are now actually removed from the weight tensor
print(layer.weight)

# Dynamic quantization for the model
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Define the student model
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # Assume binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# Assume teacher_model is your pre-trained large language model
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# Assume input_data, attention_mask, and labels are your input tensors
input_data = torch.tensor([[101, 102, 103, 104, 105]])  # Replace with actual data
attention_mask = torch.tensor([[1, 1, 1, 1, 1]])  # Replace with actual data
labels = torch.tensor([0])  # Replace with actual data

# Forward pass through the teacher model
with torch.no_grad():
    teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

# Forward pass through the student model
student_logits = student_model(input_data, attention_mask=attention_mask)

# Calculate the distillation loss
distillation_loss = nn.KLDivLoss()(F.log_softmax(student_logits / 2, dim=-1),
                                   F.softmax(teacher_logits / 2, dim=-1))

# You might also want to add a classification loss term
classification_loss = nn.CrossEntropyLoss()(student_logits, labels)

# Total loss
loss = distillation_loss + classification_loss

# Backward pass and optimization
optimizer = AdamW(student_model.parameters())
loss.backward()
optimizer.step()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       grad_fn=<MulBackward0>)
Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0000],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0000,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0000, -0.0952,  0.0000,  ..., -0.0191, -0.0508, -0.0000]],
       requires_grad=True)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 2 التاكد من نقل المعرفة

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertForSequenceClassification, AdamW

# 1. تحقق من التقطير (Pruning)
def verify_pruning(model, layer_path):
    # تحديد الطبقة المطلوبة
    layer = eval(f"model.{layer_path}")

    # التأكد من وجود أوزان مقطرة (أوزان بقيمة صفر)
    pruned_weights = (layer.weight == 0).sum().item()
    total_weights = layer.weight.numel()

    print(f"عدد الأوزان المقطرة: {pruned_weights} من أصل {total_weights}")
    print(f"نسبة التقطير: {pruned_weights / total_weights * 100:.2f}%")

# 2. تحقق من نقل المعرفة (Knowledge Distillation)
def verify_distillation(teacher_model, student_model, input_data, attention_mask):
    # وضع النموذجين في وضع التقييم (Evaluation Mode)
    teacher_model.eval()
    student_model.eval()

    # تمرير البيانات عبر نموذج المعلم
    with torch.no_grad():
        teacher_logits = teacher_model(input_data, attention_mask=attention_mask).logits

    # تمرير البيانات عبر نموذج الطالب
    student_logits = student_model(input_data, attention_mask=attention_mask)

    # حساب التشابه بين خرج النموذجين باستخدام Cosine Similarity
    cosine_sim = F.cosine_similarity(teacher_logits.flatten(), student_logits.flatten(), dim=0).item()

    print(f"التشابه بين خرج النموذجين (Cosine Similarity): {cosine_sim:.4f}")

# 3. إنشاء نموذج الطالب
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, 2)  # افترض تصنيف ثنائي

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        return logits

# 4. تحميل النماذج
teacher_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
student_model = StudentModel()

# 5. بيانات عشوائية للاختبار
input_data = torch.tensor([[101, 102, 103, 104, 105]])  # بيانات عشوائية (يجب استبدالها ببيانات فعلية)
attention_mask = torch.tensor([[1, 1, 1, 1, 1]])  # قناع الانتباه (يجب استبدالها ببيانات فعلية)

# 6. تطبيق التقطير على نموذج الطالب
import torch.nn.utils.prune as prune
layer_to_prune = "bert.encoder.layer[0].attention.self.query"
prune.l1_unstructured(eval(f"student_model.{layer_to_prune}"), name='weight', amount=0.2)

# 7. التحقق من التقطير
print("التحقق من التقطير:")
verify_pruning(student_model, layer_to_prune)

# 8. التحقق من نقل المعرفة
print("\nالتحقق من نقل المعرفة:")
verify_distillation(teacher_model, student_model, input_data, attention_mask)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


التحقق من التقطير:
عدد الأوزان المقطرة: 117965 من أصل 589824
نسبة التقطير: 20.00%

التحقق من نقل المعرفة:
التشابه بين خرج النموذجين (Cosine Similarity): 0.9612


In [3]:
!rm -rf /root/.cache